In [28]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from gsmmutils.model.COBRAmodel import MyModel, check_balance
from cobra.flux_analysis import find_blocked_reactions, pfba, flux_variability_analysis
from cobra.io import write_sbml_model, read_sbml_model
from gsmmutils.omics.omics_integration import OmicsIntegration 
import matplotlib.pyplot as plt
import pandas as pd
from os.path import join
from mewpy.omics import eFlux,ExpressionSet
import math
import numpy as np
from cobra.sampling import ACHRSampler
DATA_PATH = "/home/ecunha/omics-integration/data/dsalina/PRJNA437866"

In [19]:
model = MyModel(join(DATA_PATH, "../models/model_ds.xml"), "e_Biomass__cytop")
omics = OmicsIntegration(join(DATA_PATH, 'raw_counts.txt'), samples_names={"SRR6825159_Aligned.sortedByCoord.out.bam": "control_1",
                                                                    "SRR6825160_Aligned.sortedByCoord.out.bam": "control_2",
                                                                    "SRR6825161_Aligned.sortedByCoord.out.bam": "control_3",
                                                                    "SRR6825162_Aligned.sortedByCoord.out.bam": "nacl_1",
                                                                    "SRR6825163_Aligned.sortedByCoord.out.bam": "nacl_2",
                                                                    "SRR6825164_Aligned.sortedByCoord.out.bam": "nacl_3",
                                                                    "SRR6825165_Aligned.sortedByCoord.out.bam": "h2o2_1",
                                                                    "SRR6825166_Aligned.sortedByCoord.out.bam": "h2o2_2",
                                                                    "SRR6825167_Aligned.sortedByCoord.out.bam": "h2o2_3",
                                                                    "SRR6825168_Aligned.sortedByCoord.out.bam": "sorb_1",
                                                                    "SRR6825169_Aligned.sortedByCoord.out.bam": "sorb_2",
                                                                    "SRR6825170_Aligned.sortedByCoord.out.bam": "sorb_3",
                                                                    }, model=model)

omics.getmm = pd.read_csv(join(DATA_PATH, "getmm_with_replicates.tsv"), sep="\t", index_col=0)
omics.getmm.head()

Loading

Reactions: 4504
Metabolites: 3666
Genes: 1696
Model loaded


,control_1,control_2,control_3,h2o2_1,h2o2_2,h2o2_3,nacl_1,nacl_2,nacl_3,sorb_1,sorb_2,sorb_3
GeneID,,,,,,,,,,,,
KAF5843879_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KAF5843880_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KAF5843881_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KAF5843882_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KAF5843883_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
omics.getmm.head()

,GeneID,control_1,control_2,control_3,h2o2_1,h2o2_2,h2o2_3,nacl_1,nacl_2,nacl_3,sorb_1,sorb_2,sorb_3
0,KAF5843879_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,KAF5843880_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,KAF5843881_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,KAF5843882_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,KAF5843883_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
omics.sum_tech_reps()
omics_data = omics.getmm.applymap(lambda x: math.log2(x + 1))

/home/ecunha/miniforge3/envs/omics-integration/lib/python3.10/site-packages/gsmmutils-0.0.2-py3.10.egg/gsmmutils/omics/omics_integration.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.counts.loc[:, group] = self.counts.iloc[:, idx].sum(axis=1).values
/home/ecunha/miniforge3/envs/omics-integration/lib/python3.10/site-packages/gsmmutils-0.0.2-py3.10.egg/gsmmutils/omics/omics_integration.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.counts.loc[:, group] = self.counts.iloc[:, idx].sum(a

In [66]:
expression = omics_data["control"].to_numpy()[:, np.newaxis]
set_expression = ExpressionSet(omics_data.index.to_list(), conditions=["control"], expression=expression)

In [128]:
res = eFlux(model, set_expression, scale_rxn = "R00024__chlo",
            scale_value=100, constraints = {"e_Biomass__cytop": 0.10})

Read LP format model from file /tmp/tmpu8bl1833.lp
Reading time = 0.07 seconds
: 3666 rows, 9008 columns, 38822 nonzeros


In [130]:
for key, value in res.fluxes.items():
    if value != 0:    
        print(key, value)

In [131]:
res.dataframe

,Flux rate
Reaction ID,
